In [34]:
import pandas as pd
import numpy as np
import string

data_frame = pd.read_csv("SMSSpamCollection", sep="\t" ,header = None, names=["label","message"])
display(data_frame)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [62]:
total = data_frame.shape[0]
ham = data_frame.loc[data_frame['label'] == 'ham']
ham_counts = ham.shape[0]
spam = data_frame.loc[data_frame['label'] == 'spam']
spam_counts = spam.shape[0]

In [63]:
print('total documents: ', total, 'lines')
print('ham: ', ham_counts, 'lines')
print('spam: ', spam_counts, 'lines')

total documents:  5572 lines
ham:  4825 lines
spam:  747 lines


In [29]:
print(data_frame['label'].value_counts())

label
ham     2
spam    1
Name: count, dtype: int64


In [65]:
i = 1
for message in data_frame['message']:
    words = message.split()
    total_words = len(words)
    print('message ',i,': ',total_words)
    i+=1

message  1 :  20
message  2 :  6
message  3 :  28
message  4 :  11
message  5 :  13
message  6 :  32
message  7 :  16
message  8 :  26
message  9 :  26
message  10 :  29
message  11 :  21
message  12 :  26
message  13 :  26
message  14 :  37
message  15 :  8
message  16 :  19
message  17 :  4
message  18 :  19
message  19 :  13
message  20 :  24
message  21 :  8
message  22 :  11
message  23 :  11
message  24 :  20
message  25 :  11
message  26 :  28
message  27 :  5
message  28 :  32
message  29 :  15
message  30 :  12
message  31 :  26
message  32 :  41
message  33 :  6
message  34 :  17
message  35 :  29
message  36 :  28
message  37 :  9
message  38 :  8
message  39 :  4
message  40 :  30
message  41 :  17
message  42 :  34
message  43 :  33
message  44 :  4
message  45 :  13
message  46 :  3
message  47 :  8
message  48 :  5
message  49 :  14
message  50 :  30
message  51 :  12
message  52 :  23
message  53 :  16
message  54 :  55
message  55 :  22
message  56 :  13
message  57 : 

In [77]:
stop_words = set(stopwords.words('english'))
numbers = set('0123456789')
links = ['www', 'http']
tags = ['@', '#']
punctuation = set(string.punctuation) - {'@', '#'}

def preprocess1(message):
    message = message.lower()
    message = ''.join(char for char in message if char not in punctuation)
    words = message.split()
    preprocess_words = []
    for word in words:
        if (
            word not in stop_words
            and len(word) >= 3
            and not any(char.isdigit() for char in word)
            and not any(link in word for link in links)
            and not any(tag in word for tag in tags)
        ):
            preprocess_words.append(word)
        
        
    return preprocess_words

In [35]:
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

def remove_urls(text):
    return re.sub(r'http\S+|www\.\S+','', text)

def remove_tags(text):
    return re.sub(r'#\w+|@\w+','', text)

def remove_punctuation(text):
    text = ''.join(char for char in text if char not in string.punctuation)
    return text

def remove_numbers(text):
    return re.sub(r'\d+','',text)

def remove_repetitions(text):
    return re.sub(r'(.)\1{2,}',r'\1\1', text)

def remove_small(text):
    return [word for word in text if len(word) >= 3]

data_frame['message'] = data_frame['message'].astype(str).str.lower()
data_frame['message'] = data_frame['message'].apply(remove_urls)
data_frame['message'] = data_frame['message'].apply(remove_tags)
data_frame['message'] = data_frame['message'].apply(remove_punctuation)
data_frame['message'] = data_frame['message'].apply(remove_numbers)
data_frame['message'] = data_frame['message'].apply(remove_repetitions)
data_frame['message'] = data_frame['message'].apply(lambda x: x.split())
data_frame['message'] = data_frame['message'].apply(remove_stopwords)
data_frame['message'] = data_frame['message'].apply(remove_small)

display(data_frame)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label,message
0,ham,"[jurong, point, crazy, available, bugis, great..."
1,ham,"[lar, joking, wif, oni]"
2,spam,"[free, entry, wkly, comp, win, cup, final, tkt..."
3,ham,"[dun, say, early, hor, already, say]"
4,ham,"[nah, dont, think, goes, usf, lives, around, t..."
...,...,...
5567,spam,"[time, tried, contact, pound, prize, claim, ea..."
5568,ham,"[going, esplanade, home]"
5569,ham,"[pity, mood, soany, suggestions]"
5570,ham,"[guy, bitching, acted, like, interested, buyin..."


In [36]:
#documents = []
#for message in data_frame['message']:
#    words = preprocess(message)
#    print(words)
#    documents.append(words)



documents = []
for doc in data_frame['message']:
    documents.append(doc)

vocab = set()
for doc in documents:
    vocab.update(doc)
vocab = sorted(list(vocab))

vocab_index = {}
for i in range(len(vocab)):
    vocab_index[vocab[i]] = i

print(documents)
print("vocab: ", vocab)
print("vocab_index: ", vocab_index)

def td(doc):
    td_vector = np.zeros(len(vocab))
    for word in doc:
        td_vector[vocab_index[word]] +=1
    td_vector = td_vector/len(doc)
    return td_vector

def idf(documents):
    idf = np.zeros(len(vocab))
    for doc in documents:
        seen = set()
        for word in doc:
            if word not in seen:
                idf[vocab_index[word]] += 1
                seen.add(word)
    print("idf:" , idf)
    idf = np.log(len(documents)/(idf))
    return idf

idf_vector = idf(documents)
td_table = np.array([td(doc) for doc in documents])

print(td_table)

td_idf = td_table * idf_vector
td_idf_df = pd.DataFrame(td_idf, columns = vocab)
display(td_idf_df)



[['jurong', 'point', 'crazy', 'available', 'bugis', 'great', 'world', 'buffet', 'cine', 'got', 'amore', 'wat'], ['lar', 'joking', 'wif', 'oni'], ['free', 'entry', 'wkly', 'comp', 'win', 'cup', 'final', 'tkts', 'may', 'text', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', 'overs'], ['dun', 'say', 'early', 'hor', 'already', 'say'], ['nah', 'dont', 'think', 'goes', 'usf', 'lives', 'around', 'though'], ['freemsg', 'hey', 'darling', 'weeks', 'word', 'back', 'like', 'fun', 'still', 'std', 'chgs', 'send', 'rcv'], ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aids', 'patent'], ['per', 'request', 'melle', 'melle', 'oru', 'minnaminunginte', 'nurungu', 'vettam', 'set', 'callertune', 'callers', 'press', 'copy', 'friends', 'callertune'], ['winner', 'valued', 'network', 'customer', 'selected', 'receivea', 'prize', 'reward', 'claim', 'call', 'claim', 'code', 'valid', 'hours'], ['mobile', 'months', 'entitled', 'update', 'latest', 'colour', 'mobiles', 'camera', 'free', 'call', '

C:\Users\maria\AppData\Local\Temp\ipykernel_8120\3320162588.py:30: RuntimeWarning: invalid value encountered in divide
  td_vector = td_vector/len(doc)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


,aah,aaniye,aaooright,aathilove,aathiwhere,abbey,abdomen,abeg,abelu,aberdeen,...,£msg,£perweeksub,£perwksub,£pm,£pmmorefrommobilebremovedmobypoboxlsyf,£week,£wk,üll,“harry,…thanks
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
